In [1]:
cd /private/home/dpf/projects/codex

/private/home/dpf/projects/codex


In [2]:
from models import PLBart
from models import add_model_args
from argparse import ArgumentParser
parser = ArgumentParser()
add_model_args(parser)
args = parser.parse_args([])
print(args)

INFO:numexpr.utils:Note: detected 80 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 80 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


Namespace(batch_size=1, beam=1, candidate_scoring='mean', max_tokens=450, model_name=None, prompt_prefix=None, temperature=0.6, tokenizer_name=None, top_p=0.95)


In [3]:
model = PLBart(args)

2022-07-31 15:06:42.471840: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/public/slurm/20.11.8/lib
2022-07-31 15:06:42.471870: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
model.lm_model.device

device(type='cuda', index=0)

In [5]:
from infill_evaluation import generate_he_infill_problems
problems = list(generate_he_infill_problems(args, "one_line"))

In [6]:
from models import TruncationParameters

In [23]:
infilling_problem = problems[0][1][-1]
parts = infilling_problem["prompt_parts"]
prefix, suffix = parts
truncation_parameters = [
    TruncationParameters.from_heuristics(["num_lines"], infilling_problem["missing_lines"], suffix)
]
print(parts)

['from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n', '']


In [24]:
parts

['from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n',
 '']

In [25]:
results = model.infill(parts, truncation_parameters, sampling=True, temperature=0.2, top_p=0.95, true_infill=infilling_problem['missing_lines'], line_aligned=True)

prefix_pretokens
from typing import List NEW_LINE def has_close_elements ( numbers : List [ float ] , threshold : float ) -> bool : NEW_LINE INDENT """ ▁ Check ▁ if ▁ in ▁ given ▁ list ▁ of ▁ numbers , ▁ are ▁ any ▁ two ▁ numbers ▁ closer ▁ to ▁ each ▁ other ▁ than STRNEWLINE ▁ ▁ ▁ ▁ given ▁ threshold . STRNEWLINE ▁ ▁ ▁ ▁ > > > ▁ has _ close _ elements ( [1.0 , ▁ 2.0 , ▁ 3.0 ] , ▁ 0.5 ) STRNEWLINE ▁ ▁ ▁ ▁ False STRNEWLINE ▁ ▁ ▁ ▁ > > > ▁ has _ close _ elements ( [1.0 , ▁ 2.8 , ▁ 3.0 , ▁ 4.0 , ▁ 5.0 , ▁ 2.0 ] , ▁ 0.3 ) STRNEWLINE ▁ ▁ ▁ ▁ True STRNEWLINE ▁ ▁ ▁ ▁ """ NEW_LINE for idx , elem in enumerate ( numbers ) : NEW_LINE INDENT for idx2 , elem2 in enumerate ( numbers ) : NEW_LINE INDENT if idx != idx2 : NEW_LINE INDENT distance = abs ( elem - elem2 ) NEW_LINE if distance < threshold : NEW_LINE INDENT return True
prefix_infill_pretokens
from typing import List NEW_LINE def has_close_elements ( numbers : List [ float ] , threshold : float ) -> bool : NEW_LINE INDENT """ ▁ Check ▁ if ▁ 

In [27]:
infilling_problem['missing_lines']

'    return False'

In [28]:
results['choices'][0]['complete'][1]

'    return False'

In [29]:
print(parts[-1])

In [30]:
print(parts[0])
print("-----")
print(parts[1])

from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """
    for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(numbers):
            if idx != idx2:
                distance = abs(elem - elem2)
                if distance < threshold:
                    return True

-----



In [31]:
print(results['choices'][0]['generated_text'][0])

python<s> def has_close_elements ( numbers : List [ float ] ) -> bool : NEW_LINE INDENT from typing import List NEW_LINE def has_close_elements ( numbers : List [ float ], threshold : float ) -> bool : NEW_LINE INDENT """ Check if in given list of numbers, are any two numbers closer to each other than STRNEWLINE given threshold. STRNEWLINE > > > has _ close _ elements ( [1.0, 2.0, 3.0 ], 0.5 ) STRNEWLINE False STRNEWLINE > > > has _ close _ elements ( [1.0, 2.8, 3.0, 4.0, 5.0, 2.0 ], 0.3 ) STRNEWLINE True STRNEWLINE """ NEW_LINE for idx, elem in enumerate ( numbers ) : NEW_LINE INDENT for idx2, elem2 in enumerate ( numbers ) : NEW_LINE INDENT if idx!= idx2 : NEW_LINE INDENT distance = abs ( elem - elem2 ) NEW_LINE if distance < threshold : NEW_LINE INDENT return True NEW_LINE DEDENT DEDENT DEDENT DEDENT return False NEW_LINE DEDENT return has_close_elements NEW_LINE DEDENT</s>


In [15]:
from plbart_utils import detokenize_code

In [16]:
print("\n".join(results['choices'][0]['complete']))

from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """

    for idx, elem in enumerate ( numbers ) :
        for idx2, elem2 in enumerate(numbers):
            if idx != idx2:
                distance = abs(elem - elem2)
                if distance < threshold:
                    return True

    return False


In [32]:
from plbart_utils import detokenize_code
print("--input--")
print(results['choices'][0]['input_text'][0])
print("--output--")
print(results['choices'][0]['generated_text'][0])
print("--parsed output--")
print(detokenize_code(results['choices'][0]['generated_text'])[0])
# print(results['choices'][0]['generated_text'])

--input--
from typing import List NEW_LINE def has_close_elements ( numbers : List [ float ] , threshold : float ) -> bool : NEW_LINE INDENT """ ▁ Check ▁ if ▁ in ▁ given ▁ list ▁ of ▁ numbers , ▁ are ▁ any ▁ two ▁ numbers ▁ closer ▁ to ▁ each ▁ other ▁ than STRNEWLINE ▁ ▁ ▁ ▁ given ▁ threshold . STRNEWLINE ▁ ▁ ▁ ▁ > > > ▁ has _ close _ elements ( [1.0 , ▁ 2.0 , ▁ 3.0 ] , ▁ 0.5 ) STRNEWLINE ▁ ▁ ▁ ▁ False STRNEWLINE ▁ ▁ ▁ ▁ > > > ▁ has _ close _ elements ( [1.0 , ▁ 2.8 , ▁ 3.0 , ▁ 4.0 , ▁ 5.0 , ▁ 2.0 ] , ▁ 0.3 ) STRNEWLINE ▁ ▁ ▁ ▁ True STRNEWLINE ▁ ▁ ▁ ▁ """ NEW_LINE INDENT <mask> NEW_LINE NEW_LINE INDENT for idx2 , elem2 in enumerate ( numbers ) : NEW_LINE INDENT if idx != idx2 : NEW_LINE INDENT distance = abs ( elem - elem2 ) NEW_LINE if distance < threshold : NEW_LINE INDENT return True NEW_LINE DEDENT DEDENT DEDENT DEDENT return False NEW_LINE DEDENT
--output--
python<s> typing import List NEW_LINE def has_close_elements ( numbers : List [ float ], threshold : float ) -> bool : NEW_

In [20]:
# print('\n'.join(results['choices'][0]['complete']))
print(results['choices'][0]['complete'][2])

                if distance < threshold:
                    return True

    return False


In [ ]:
print(parts[0])
print("-----")
print(parts[1])

from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """
    for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(numbers):
            if idx != idx2:

-----
                if distance < threshold:
                    return True

    return False
